In [29]:
import pandas as pd
import boto3
import sys
from config import variables
from tools.s3_helper import s3
from tools.spotify_helper import SpotipyHelper

s3_ = s3()
spotify_ = SpotipyHelper()
bucket = variables.BUCKET_COUNTRY
file_names = s3_.read_bucket_file_names(bucket)

In [30]:
file_names[-20:]

['Turks and Caicos Islands',
 'Tuvalu',
 'Uganda',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United States Minor Outlying Islands',
 'United States',
 'Uruguay',
 'Uzbekistan',
 'Vanuatu',
 'Venezuela, Bolivarian Republic of',
 'Viet Nam',
 'Virgin Islands, British',
 'Wallis and Futuna',
 'Western Sahara',
 'Yemen',
 'Zambia',
 'Zimbabwe',
 'Åland Islands']

In [27]:
name = 'Virgin Islands, U'
obj = s3_.client.get_object(Bucket= bucket, Key= f'{name}.csv') 
df = pd.read_csv(obj['Body']) 

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [13]:
df.columns = [c.lower() for c in df.columns]
df.columns

Index(['track number', 'track', 'artist', 'uri', 'genre'], dtype='object')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   track number  0 non-null      object
 1   track         0 non-null      object
 2   artist        0 non-null      object
 3   uri           0 non-null      object
 4   genre         0 non-null      object
dtypes: object(5)
memory usage: 0.0+ bytes


In [15]:
df.nunique()

track number    0
track           0
artist          0
uri             0
genre           0
dtype: int64

In [16]:
df.drop_duplicates(subset=['uri'],inplace=True)

,track number,track,artist,uri,genre


In [17]:
uris = list(df.uri.values)
len(uris)

0

In [18]:
features = []
for i in range(0,len(uris),50):
    sebset = uris[i:i+50]
    features.append(spotify_.get_audio_features(sebset))
 

In [19]:

features_df = pd.DataFrame()
for i in range(0,len(features)):
    if features[i]:
        temp = pd.DataFrame(features[i])
        features_df = features_df.append(temp,ignore_index=True)

full_df = df.merge(features_df, how='left',on='uri')


with open(f'/tmp/temp.csv','w') as file:
    full_df.to_csv(file)
s3_.save_data_s3('temp',bucket,f'{name}-featues.csv')    


KeyError: 'uri'